In [1]:
import pandas as pd
import numpy as np

In [2]:
#1 задание
#файл пришел без заголовков
#сразу воспользуемся аргументом для замены пользовательского значения место пропуска
missing_values = ['?']
origin_horses = pd.read_csv('horse_data.csv', header=None, na_values = missing_values)

In [3]:
#создаю новый дф с нужными столбцами
horses = origin_horses[[0, 1, 3, 4, 5, 6, 10, 23]]
horses.columns = ['surgery', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']

In [4]:
horses

,surgery,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2
1,1.0,1,39.2,88.0,20.0,NaN,3.0,2
2,2.0,1,38.3,40.0,24.0,1.0,3.0,2
3,1.0,9,39.1,164.0,84.0,4.0,2.0,1
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,2
296,2.0,1,37.2,72.0,24.0,3.0,4.0,1
297,1.0,1,37.5,72.0,30.0,4.0,4.0,1
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1


In [5]:
#2 задание
# считаем базовые статистики
horses.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,300.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.363333
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.481763
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,2.000000


In [6]:
# describe не считает моду
horses.mode()

,surgery,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,38.0,48.0,20.0,3.0,3.0,1


In [7]:
#находим выбросы
# для столбца Surgery не имеет смысл искать выбросы,так как значения принимают только 3 варианта, один из которых пустое значение
horses['surgery'].unique()

array([ 2.,  1., nan])

In [8]:
# столбец Age имеет только 2 значения, нет смысла искать выбросы
horses['Age'].unique()

array([1, 9], dtype=int64)

In [9]:
# столбец rectal tempreture, в столбце 40 значений, возможно среди них есть выбросы
horses['rectal temperature'].nunique()

40

In [54]:
#имем выбросы через межквартильный размах, как видно, почти 25 процентов являются выбросами,
#что возможно говорит о том, что четверть лошадей имеют повышенную температуру при кишечных коликах
q1 = horses['rectal temperature'].quantile(0.25)
q3 = horses['rectal temperature'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - (1.5 * IQR)
upper_bound = q3 + (1.5 * IQR)
horses['rectal temperature'].between(lower_bound, upper_bound, inclusive=True).value_counts(normalize=True)

True     0.753333
False    0.246667
Name: rectal temperature, dtype: float64

In [ ]:
#столбец pulse, в столбце 52 значения, имеет смысл найти выбросы
horses['pulse'].nunique()

In [95]:
#как видно, только около 10 процентов относятся к выбросам, можно предположить, что кишечные колики не оказывают влияние на пулс лошадей
q1 = horses['pulse'].quantile(0.25)
q3 = horses['pulse'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - (1.5 * IQR)
upper_bound = q3 + (1.5 * IQR)
horses['pulse'].between(lower_bound, upper_bound, inclusive=True).value_counts(normalize=True)

True     0.903333
False    0.096667
Name: pulse, dtype: float64

In [73]:
#в столбце частоты дыхания 40 различных значений, имеет смысл проверить на выбросы
horses['respiratory rate'].nunique()

40

In [96]:
# как видно, выбросы составляют четверть, что схоже с показателями пульса, как известно, пульс и частота дыхания взаимосвязаны,
# в дальнейшем имеет смысл узнать их пересечения.
q1 = horses['respiratory rate'].quantile(0.25)
q3 = horses['respiratory rate'].quantile(0.75)
IQR = q3 - q1
lower_bound = q1 - (1.5 * IQR)
upper_bound = q3 + (1.5 * IQR)
horses['respiratory rate'].between(lower_bound, upper_bound, inclusive=True).value_counts(normalize=True)

True     0.75
False    0.25
Name: respiratory rate, dtype: float64

In [97]:
#в данном столбце имеются 4 показателя, 3 и 4 говорят о том, что данное состояние может вызвать шок
#как видно, чуть больше половины лошадей имеют температуру конечностей ниже необходимой
horses['temperature of extremities'].value_counts(normalize=True)

3.0    0.446721
1.0    0.319672
2.0    0.122951
4.0    0.110656
Name: temperature of extremities, dtype: float64

In [98]:
# в столбце 5 уровней градации по уровню боли, как видно примерно 50% имеют средний уровень, стоит отметить, что сильную боль испытывают около 15% лошадей
horses['pain'].value_counts(normalize=True)

3.0    0.273469
2.0    0.240816
5.0    0.171429
4.0    0.159184
1.0    0.155102
Name: pain, dtype: float64

In [ ]:
#в столбце имеется 2 состояния из 3, так как ни одну лошадь не усыпили
# как видно, около 64% лошадей с кишечными коликами живы
horses['outcome'].value_counts(normalize=True)

In [142]:
#3 задание
#для того, чтобы заполнить пропуски надо посмотреть их процент, приходящийся на каждый столбец
(horses.isna().mean() * 100).round(2)

surgery                        0.00
Age                            0.00
rectal temperature            20.07
pulse                          8.03
respiratory rate              19.40
temperature of extremities    18.73
pain                          18.39
outcome                        0.00
dtype: float64

In [165]:
#в surgery всего 3 процента пропуска,а в Pulse 8%, эти значения можно удалить
drop_horses = horses.dropna(subset=['surgery','pulse'])

In [166]:
#их удаление почти никак не повлияли на базовые статистики
drop_horses.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,275.000000,275.000000,233.000000,275.000000,235.000000,228.000000,228.000000,275.000000
mean,1.403636,1.640000,38.152790,72.000000,30.629787,2.315789,2.881579,1.374545
std,0.491521,2.174302,0.729964,28.646219,17.808909,1.052087,1.286669,0.484888
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,28.000000,3.000000,3.000000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.000000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.000000,2.000000


In [168]:
(drop_horses.isna().mean() * 100).round(2)

surgery                        0.00
Age                            0.00
rectal temperature            15.27
pulse                          0.00
respiratory rate              14.55
temperature of extremities    17.09
pain                          17.09
outcome                        0.00
dtype: float64

In [169]:
#скорее всего есть взаимосвязи между температурой, пульсом и частотой дыхания. Как проверить?
#Температра конечностей и боль скорее всего не влияют на другие переменные
#Пока заполняем все оставшиеся пропуски медианой или средним

In [170]:
drop_horses['rectal temperature'].fillna(drop_horses['rectal temperature'].mean(), inplace = True)

C:\Users\sasha\Anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [172]:
drop_horses['respiratory rate'].fillna(drop_horses['respiratory rate'].median(), inplace = True)

In [173]:
drop_horses['temperature of extremities'].fillna(drop_horses['temperature of extremities'].median(), inplace = True)

In [174]:
drop_horses['pain'].fillna(drop_horses['pain'].mean(), inplace = True)

In [175]:
#по итогу базовые статистики не сильно изменились
drop_horses.describe()

,surgery,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
count,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000
mean,1.403636,1.640000,38.152790,72.000000,30.247273,2.432727,2.881579,1.374545
std,0.491521,2.174302,0.671692,28.646219,16.483921,0.991764,1.171129,0.484888
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,20.000000,1.000000,2.000000,1.000000
50%,1.000000,1.000000,38.152790,64.000000,28.000000,3.000000,2.881579,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,3.500000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.000000,2.000000


In [176]:
drop_horses.isna().sum()

surgery                       0
Age                           0
rectal temperature            0
pulse                         0
respiratory rate              0
temperature of extremities    0
pain                          0
outcome                       0
dtype: int64